In [19]:
import numpy as np
import math
import pygame as pg

In [20]:
#===========CHANGABLE PARAMETERS=================
REWARD     = 1
PENALTY    = -1
DISCOUNT   = 1
LEARN_RATE = 1


#===========DEFINE CONSTANTS AND DICS=============
WALL_LEN = 1
PADDLE_H = 0.2
MIN_V = 0.03

init_state = (0.5, 0.5, 0.03, 0.01, 0.5 - PADDLE_H / 2)

ACTION_DIC={'UP':-1,'STAY':0,'DOWN':1}

BOARD_SIZE = 12
X_VBALL_DIS = [-1,1]
Y_VBALL_DIS = [-1,0,1]

PADDLE_SPACE = 12

STATE_SPACE = (BOARD_SIZE,BOARD_SIZE,len(X_VBALL_DIS),len(Y_VBALL_DIS),PADDLE_SPACE)

X_V_TSH = 0.03
Y_V_TSH = 0.015

State: A tuple (ball_x, ball_y, velocity_x, velocity_y, paddle_y).

ball_x and ball_y are real numbers on the interval [0,1]. The lines x=0, y=0, and y=1 are walls; the ball bounces off a wall whenever it hits. The line x=1 is defended by your paddle.

The absolute value of velocity_x is at least 0.03, which guarantees that the ball is moving either left or right at a reasonable speed.

paddle_y represents the top of the paddle (the side closer to y=0) and is on the interval [0, 1 - paddle_height], where paddle_height = 0.2, as can be seen in the image above. (The x-coordinate of the paddle is always paddle_x=1, so you do not need to include this variable as part of the state definition).


In [23]:
#==============DEFINE STATE CLASS===============
class state:
    
    def __init__(self,ball_x,ball_y,velocity_x,velocity_y,paddle_y):
        self.ball_x = ball_x              #real numbers on the interval [0,1]
        self.ball_y = ball_y
        self.velocity_x = velocity_x
        self.velocity_y = velocity_y
        self.paddle_y = paddle_y
        self.state_tuple = (ball_x,ball_y,velocity_x,velocity_y,paddle_y)
        self._extract()
        
    def _extract(self):
        self.x_grid = math.floor(12*self.ball_x)
        self.y_grid = math.floor(12*self.ball_y)
        if(self.velocity_x>0): 
            self.x_v_sign = 1
        else: 
            self.x_v_sign = -1
            
        if(self.velocity_y>0):
            self.y_v_sign = 1
        else: 
            self.y_v_sign = -1
        
        self.paddle_grid = math.floor(12 * self.paddle_y / (1 - PADDLE_H))
        if self.ball_x>1:
            self.end_state = 1

**Actions**: Your agent's actions are chosen from the set {nothing, paddle_y += 0.04, paddle_y -= 0.04}. 
In other words, your agent can either move the paddle up, down, or make it stay in the same place. 
If the agent tries to move the paddle too high, so that the top goes off the screen, simply assign paddle_y = 0.
Likewise, if the agent tries to move any part of the paddle off the bottom of the screen, assign paddle_y = 1 - paddle_height.

**Rewards**: +1 when your action results in rebounding the ball with your paddle, -1 when the ball has passed your agent's paddle, or 0 otherwise.

**Initial State**: Use (0.5, 0.5, 0.03, 0.01, 0.5 - paddle_height / 2) as your initial state (see the state representation above). This represents the ball starting in the center and moving towards your agent in a downward trajectory, where the agent's paddle starts in the middle of the screen.

**Termination**: Consider a state terminal if the ball's x-coordinate is greater than that of your paddle, i.e., the ball has passed your paddle and is moving away from you.

We need to simulate the environment. The ball is a single point, and your paddle is a line. Therefore, you don't need to worry about the ball bouncing off the ends of the paddle. At each time step, you must:

Update the paddle position based on the action chosen by your agent.
Increment ball_x by velocity_x and ball_y by velocity_y.
Bounce:
If ball_y < 0 (the ball is off the top of the screen), assign ball_y = -ball_y and velocity_y = -velocity_y.
If ball_y > 1 (the ball is off the bottom of the screen), let ball_y = 2 - ball_y and velocity_y = -velocity_y.
If ball_x < 0 (the ball is off the left edge of the screen), assign ball_x = -ball_x and velocity_x = -velocity_x.
If moving the ball to the new coordinates resulted in the ball bouncing off the paddle, handle the ball's bounce by assigning ball_x = 2 * paddle_x - ball_x. Furthermore, when the ball bounces off a paddle, randomize the velocities slightly by using the equation velocity_x = -velocity_x + U and velocity_y = velocity_y + V, where U is chosen uniformly on [-0.015, 0.015] and V is chosen uniformly on [-0.03, 0.03]. As specified above, make sure that all |velocity_x| > 0.03.

Note: In rare circumstances, either of the velocities may increase above 1. In these cases, after applying the bounce equations given above, the ball may still be out of bounds. If this poses a problem for your implementation, feel free to impose the restriction that |velocity_x| < 1 and |velocity_y| < 1.


In [17]:
#=======DEFINE MORE HELPER FUNCTIONS===========
def bounce():
    pass

def check_termination():
    pass

def reward():
    pass

def take_action():
    pass


Instructions only for Part 1: Because this state space is continuous, to allow for it to be learned with Q-learning, we need to be able to convert the continuous state space into a discrete, finite state space. To do this, please follow the instructions below. Note that the constants may be different for the next part of the assignment, so it would be wise to define these as constants in a separate part of your code so you can change them easily. You will learn how to deal with continuous state spaces in part 2.

Treat the entire board as a 12x12 grid, and let two states be considered the same if the ball lies within the same cell in this table. Therefore there are 144 possible ball locations.
Discretize the X-velocity of the ball to have only two possible values: +1 or -1 (the exact value does not matter, only the sign).
Discretize the Y-velocity of the ball to have only three possible values: +1, 0, or -1. It should map to Zero if |velocity_y| < 0.015.
Finally, to convert your paddle's location into a discrete value, use the following equation: discrete_paddle = floor(12 * paddle_y / (1 - paddle_height)). In cases where paddle_y = 1 - paddle_height, set discrete_paddle = 11. As can be seen, this discrete paddle location can take on 12 possible values.
Add one special state for all cases when the ball has passed your paddle (ball_x > 1). This special state needn't differentiate among any of the other variables listed above, i.e., as long as ball_x > 1, the game will always be in this state, regardless of the ball's velocity or the paddle's location. This is the only state with a reward of -1.
Therefore, the total size of the state space for this problem is (144)(2)(3)(12)+1 = 10369.

In [14]:
#=============DEFINE Q-AGENT CLASS==============
class q_agent:
    
    def __init__(self):
        self.q_table = np.zeros(STATE_SPACE)
        self.end_state = 0

In [18]:
#=============DEFINE TRAIN FUCNTION=============
def train():
    pass